<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/evaluation/hss/RoBERTa_HSS_augmented_DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Investigating text data augmentation using back translation for author profiling
- - - 
RoBERTa ON HSS DS EXPERIMENTS NOTEBOOK 
- - -
RoBERTa on Hate Speech Spreaders Dataset augmented with DE backtranslation.
Code by M. Siino. 

## Importing modules.

In [1]:
!pip install simpletransformers

import matplotlib.pyplot as plt
import os
import random
import re
import shutil
import string
import tensorflow as tf
import pandas as pd

import torch

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#from google.colab import files
from io import open
from numpy.random import seed
import numpy as np
from pathlib import Path
from sklearn import svm
from simpletransformers.classification import ClassificationModel, ClassificationArgs
os.environ['TF_CUDNN_DETERMINISTIC']='true'
os.environ['TF_DETERMINISTIC_OPS']='true'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## Importing DS and extract in current working directory.

In [2]:
urlTrainingSet = "https://github.com/marco-siino/DA-ESWA/raw/main/data/hss/hss-training-augmented-de.zip"
training_set = tf.keras.utils.get_file("pan21-author-profiling-training-2021-03-14-augmented.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

urlTestSet="https://github.com/marco-siino/DA-ESWA/raw/main/data/hss/hss-test-augmented-de.zip"
test_set = tf.keras.utils.get_file("pan21-author-profiling-test-without-gold-augmented.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

1000245/1000245 [==============================] - 0s 0us/step


In [3]:
training_set_dir = os.path.join(os.path.dirname(training_set), 'pan21-author-profiling-training-2021-03-14-augmented')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan21-author-profiling-test-without-gold-augmented')

## Build folders hierarchy to use Keras folders preprocessing function.

In [4]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

.config
__MACOSX
pan21-author-profiling-test-without-gold-augmented
pan21-author-profiling-test-without-gold-augmented.zip
pan21-author-profiling-training-2021-03-14-augmented
pan21-author-profiling-training-2021-03-14-augmented.zip
sample_data
test_dir_en
train_dir_en


## Set language and directory paths.


In [5]:
# Set en and es ground truth file path for train_dir. 
language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_en = truth_file_test_dir+'/'+'truth'+'.txt'

## Read truth.txt to organize training and test dataset folders.



In [6]:
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Generate full training set.



In [7]:
# Generate full randomized training set.
batch_size=1

en_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size,
    shuffle=False
    )

en_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle=False
    )

train_ds=en_train_ds.shuffle(300,seed=1, reshuffle_each_iteration=False)
test_ds=en_test_ds.shuffle(200,seed=1, reshuffle_each_iteration=False)

train_ds_size=len(train_ds)
test_ds_size=len(test_ds)

Found 200 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


## Functions to pre-process source text. 

In [8]:
# Preprocessing function to remove some noise due to the translation.
def clean_samples(input_data):
  tag_author_lang_en_removed = tf.strings.regex_replace(input_data,'<author lang="en"', '')
  tag_author_lang_en_cased_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'<Author lang="en"', '\n\n')
  tag_opening_document_miscased = tf.strings.regex_replace(tag_author_lang_en_cased_removed,'<Document>', '<document>')
  output_data = tf.strings.regex_replace(tag_opening_document_miscased,'</Document>', '</document>')
  return output_data

## Function to convert DSs to Pandas Dataframe

In [9]:
def preprocess_and_convert_ds(clean_samples):
  # Convert English dataset.
  train_df = [] # will contain text and label
  for element in train_ds:
    authorDocument=element[0]
    label=int(element[1].numpy())
    #print(authorDocument[0])
    text = clean_samples(authorDocument[0].numpy()).numpy().decode('UTF-8')
    train_df.append({
        'text':text,
        'label':label
    })
  train_df = pd.DataFrame(train_df)

  test_df = [] # will contain text and label
  for element in test_ds:
    authorDocument=element[0]
    label=int(element[1].numpy())
    #print(authorDocument[0])
    text = clean_samples(authorDocument[0].numpy()).numpy().decode('UTF-8')
    test_df.append({
        'text':text,
        'label':label
    })
  test_df = pd.DataFrame(test_df)

  return train_df, test_df


## Some parameters definition...

In [10]:
# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

num_epochs_per_run = 10
num_runs = 5

Cuda available?  True


## Get the length of the longest sample in training set. Then adapt text.



In [11]:
def preprocess_and_adapt_ts(training_set):
  # Set a large sequence length to find the longest sample in the training set.
  sequence_length = 20000
  vectorize_layer = TextVectorization(
      standardize=clean_samples,
      output_mode='int',
      output_sequence_length=sequence_length)

  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  #vectorize_layer.get_vocabulary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
  model.add(vectorize_layer)

  longest_sample_length=1

  for element in training_set:
    authorDocument=element[0]
    label=element[1]
    
    #print("Sample considered is: ", authorDocument[0].numpy())
    #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
    #print("And has label: ", label[0].numpy())

    out=model(authorDocument)
    # Convert token list to numpy array.
    token_list = out.numpy()[0]
    token_list = np.trim_zeros(token_list,'b')
    if longest_sample_length < len(token_list):
      longest_sample_length = len(token_list)

  print("Length of the longest sample is:", longest_sample_length)

  # After tokenization longest_sample_length covers all the document lenghts in our dataset.
  sequence_length = longest_sample_length

  vectorize_layer = TextVectorization(
      standardize=clean_samples,
      output_mode='int',
      output_sequence_length=sequence_length)

  # Finally adapt the vectorize layer.
  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  return vectorize_layer

## Vectorization




In [12]:
print("\n\n* * * * VECTORIZATION STARTED * * * *")

# Preprocess training set to build a dictionary.
vectorize_layer = preprocess_and_adapt_ts(train_ds)

max_features=len(vectorize_layer.get_vocabulary()) + 1
print("Vocabulary size is:", max_features)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089




* * * * VECTORIZATION STARTED * * * *
Length of the longest sample is: 8554
Vocabulary size is: 68301


## Models definition and evaluation.




In [13]:
print("\n\n***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******")
# Print a raw and a preprocessed sample.
for element in train_ds:
  authorDocument=element[0]
  label=element[1]
  
  print("Sample considered is: ", authorDocument[0])
  print("Preprocessed: ", str(clean_samples(authorDocument[0].numpy())))
  break

# # # - - - - - MODELS DEFINITION AND EVALUATION - - - - - # # #

model_args = ClassificationArgs(num_train_epochs=1, 
                                    no_save=True, 
                                    no_cache=True, 
                                    overwrite_output_dir=True)

model = ClassificationModel("roberta", 
                                'roberta-base', 
                                args = model_args, 
                                num_labels=2, 
                                use_cuda=cuda_available)

runs_accuracy = []

print("\n\n* * * * EVALUATION USING AS PREPROCESSING FUNCTION * * * *")

# Preprocess train and test set and convert to DFs.
train_df,test_df = preprocess_and_convert_ds(clean_samples)

for run in range(1,(num_runs+1)):
  epochs_accuracy=[]
  model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
  for epoch in range (0,num_epochs_per_run):
    print("\nEPOCH NUMBER: ", epoch)
    # train model
    print("\nNOW TRAIN THE MODEL.")
    model.train_model(train_df,show_running_loss=False)
    print("\nNOW EVALUATE THE TEST DF.")
    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    # Results on test set.
    print(result)
    correct_predictions = result['tp']+result['tn']
    print("Correct predictions are: ",correct_predictions)
    total_predictions = result['tp']+result['tn']+result['fp']+result['fn']
    print("Total predictions are: ",total_predictions)
    accuracy = correct_predictions/total_predictions
    print("Accuracy on test set is:",accuracy,"\n\n")
    epochs_accuracy.append(accuracy)

  print(epochs_accuracy)
  runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies are:", runs_accuracy)
print("The median is:",runs_accuracy[2])

if (runs_accuracy[2]-runs_accuracy[0])>(runs_accuracy[4]-runs_accuracy[2]):
  max_range_from_median = runs_accuracy[2]-runs_accuracy[0]
else:
  max_range_from_median = runs_accuracy[4]-runs_accuracy[2]
final_result = str(runs_accuracy[2])+" +/- "+ str(max_range_from_median)

print("RoBERTa Accuracy Score on Test set -> ",final_result)



***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******
Sample considered is:  tf.Tensor(b'   \n\t<document> #user# where was this at? </document>\n\t\t<document> rt #user#: black man,has a stern warning for blm, don\'t mess with white people \xf0\x9f\x98\xac #url# </document>\n\t\t<document> #user# a meeting of insurrectionists...i mean a biden rally </document>\n\t\t<document> #user# #user# this is lame. instead of stringing up curlers, they should be stringing up...ahem </document>\n\t\t<document> #user# this is insane. the citizens better rise up. and this is why you never willingly give up your weapons. </document>\n\t\t<document> #user# i would have wrote: \xe2\x80\x9cmy name is... and i am not a racist, but al sharpton is a racist and i will pray for him to do better\xe2\x80\x9d </document>\n\t\t<document> rt #user#: the data came from the cdc!!! are you fricken kidding me with this spin??? </document>\n\t\t<document> rt #user#: jason, who came to 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie



* * * * EVALUATION USING AS PREPROCESSING FUNCTION * * * *


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie


EPOCH NUMBER:  0

NOW TRAIN THE MODEL.


...


NOW EVALUATE THE TEST DF.


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

{'mcc': 0.14025268244765068, 'tp': 27, 'tn': 30, 'fp': 20, 'fn': 23, 'auroc': 0.5267999999999999, 'auprc': 0.5944815059879021, 'eval_loss': 1.20306029686561}
Correct predictions are:  57
Total predictions are:  100
Accuracy on test set is: 0.57 


[0.49, 0.42, 0.46, 0.53, 0.4, 0.51, 0.51, 0.52, 0.54, 0.57]


 Over all runs maximum accuracies are: [0.55, 0.57, 0.57, 0.59, 0.62]
The median is: 0.57
RoBERTa Accuracy Score on Test set ->  0.57 +/- 0.050000000000000044
